In [2]:
#addprocs(11)

In [3]:
import FunctionalCollections
import Iterators
import Pipe
import Compat
import JLD
@everywhere using FunctionalCollections
@everywhere using Iterators
@everywhere using Pipe
@everywhere using Compat
@everywhere using JLD

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

push!(LOAD_PATH, ".")
push!(LOAD_PATH, "../util/")

4-element Array{ByteString,1}:
 "/home/ubuntu/build/julia-master/usr/local/share/julia/site/v0.5"
 "/home/ubuntu/build/julia-master/usr/share/julia/site/v0.5"      
 "."                                                              
 "../util/"                                                       

In [4]:
#shuffled_indexes = 1:length(ground_sents) |> collect |> shuffle!
#nfolds=10
#fold_indexes = Vector{Int}[
#    shuffled_indexes[(ii-1)*end÷nfolds + 1: ii*end÷nfolds]
#    for ii in 1:nfolds]

#@save("brown_glove_folds.jld", fold_indexes)

In [5]:
@everywhere fold_indexes=load("brown_glove_folds.jld","fold_indexes")

@everywhere function fold_split(fold_ii, raw_bow_res)
    ground_sents = Vector{ASCIIString}[rset[1] for rset in raw_bow_res]
    reconstructed_bows = Vector{ASCIIString}[rset[2] for rset in raw_bow_res]
    
    test_indexes = fold_indexes[fold_ii]
    training_indexes = trues(ground_sents)
    training_indexes[test_indexes]=false

    test_unordered_sents = reconstructed_bows[test_indexes]
    test_ground = ground_sents[test_indexes]
    training_sents = ground_sents[training_indexes]
    test_unordered_sents,test_ground, training_sents
end
    

In [6]:
@everywhere typealias S ASCIIString
@everywhere typealias State{T} Tuple{T,T}

import PyCall
@everywhere using PyCall
#http://www.nltk.org/howto/probability.html
@everywhere @pyimport nltk
@everywhere @pyimport nltk.probability as nltk_prob

@everywhere function train_language_model{T}(train_corpus::Vector{Vector{T}})
    function py_collect(xs::PyObject)
        xst = []
        for x in xs
            push!(xst,x)
        end
        xst
    end
    function trigram_buffer(sent)
        [START_MARKER1, START_MARKER2, sent..., END_MARKER1, END_MARKER2] 
    end

    training_trigrams = vcat([py_collect(nltk.trigrams(trigram_buffer(sent))) for sent in train_corpus]...)
    kn_prob_dist = nltk_prob.KneserNeyProbDist(pycall(nltk_prob.FreqDist, PyObject, training_trigrams))
    
    function trigram_model(given1::S, given2::S, event::S)
        kn_prob_dist[:prob]((given1, given2, event))
    end
end

/usr/local/lib/python2.7/dist-packages/nltk/app/__init__.py:29: UserWarning: nltk.app package not loaded (please install Tkinter library).
  warnings.warn("nltk.app package not loaded "
/usr/local/lib/python2.7/dist-packages/nltk/draw/__init__.py:15: UserWarning: nltk.draw package not loaded (please install Tkinter library).
  warnings.warn("nltk.draw package not loaded "


In [7]:
@everywhere const START_MARKER1 = "**START1**"
@everywhere const START_MARKER2 = "**START2**"
@everywhere const END_MARKER1 = "**END1**"
@everywhere const END_MARKER2 = "**END2**"


In [8]:
@everywhere function best_order(unordered_words::Vector{S}, languauge_model::Function; beam_width=Inf)
    function transition_prob(cur_state::State{S}, next_word::S)
        languauge_model(cur_state[1],cur_state[2], next_word)
    end
        
    
    function get_options(cur_state::State{S}, remaining_words)
        if length(remaining_words)==0
            #Final Step -- leaf node
            tp = transition_prob(cur_state, END_MARKER1)
            # Given P(END_MARKER2 | curstatep[2]==END_MARKER1) = 1.0
            # Do not need to consider P(END_MARKER2 | curstatep[2]==END_MARKER1, curstatep[1])
            next_state = (cur_state[2],END_MARKER1)
            return [(tp, next_state, S[])]
        end
        #Otherwise -- nonfinal step, get all childreen
        map(1:length(remaining_words)) do ii
            next_word = remaining_words[ii]
            tp = transition_prob(cur_state, next_word)           
            still_remaining_words = sub(remaining_words,[1:ii-1; ii+1:length(remaining_words)])
            next_state = (cur_state[2],next_word)
            (tp, next_state, still_remaining_words)
        end
    end
                
    
    initial_state = (START_MARKER1, START_MARKER2)
    initial_node = (1.0, EmptyList{S}(), initial_state, unordered_words)
    pending = Collections.PriorityQueue(Tuple{Float64, FunctionalCollections.AbstractList{S}, State{S}, Vector{S}},Float64, Base.Order.Reverse)     
    pending[initial_node]=1.0
    
    #At end of search pending contains all the partway steps that have not yet been explored fully
    #But that can't be more optimal than the returned solution, we don't do anything with them right now
    while(true)
        (sofar_prob, sofar_words, state, remaining_words) = Collections.dequeue!(pending)
        
        is_terminal = state[2]==END_MARKER1
        if is_terminal
            @assert length(remaining_words)==0
            return sofar_words|> tail |> collect |> reverse!, sofar_prob #Note: We take the tail of the sofar words before reversing to cut off then end marker
        end
        
        #Nontermial, ie has children, even if it is just the single terminal child
        for (tp, next_state, still_remaining_words) in get_options(state, remaining_words)
            if tp==0.0 
                continue
            end
            total_next_prob = sofar_prob*tp
            next_sofar_words = cons(next_state[2], sofar_words)
            new_node = (total_next_prob, next_sofar_words, next_state, still_remaining_words)
            pending[new_node] = total_next_prob
        end
        if length(pending)==0
            #All Paths failed, return the best we could do so far, with the unsortable on the end
            sorted = sofar_words |> collect |> reverse!
            return ([sorted...; remaining_words...], length(sorted)*1.0)
        end
    end 
end

In [9]:
@pyimport nltk.corpus as nltk_corpus
corpus_reader=nltk_corpus.brown
corpus = Vector{ASCIIString}[[lowercase(word) for word in sent] for sent in (corpus_reader[:sents]()|> collect)]
lm = train_language_model(corpus)



trigram_model (generic function with 1 method)

In [10]:
#bow_res=load("results/bags/brown_glove300_res.jld", "res")
#@time best_order(bow_res[8][2], lm)

In [11]:
using JuMP
using CoinOptServices
using AmplNLWriter

In [37]:
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy" ,"of","manners","first","performed","in","1892", "."])
#test_bag =  shuffle(["this","is","the" ,"basis","of","a" ,"comedy", "."])
test_bag =  shuffle(["it", "is", "so", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "very", "good", "."])
#test_bag =  shuffle(["it", "is", "good", "."])
#test_bag =  shuffle(["no", "way", "."])
#test_bag =  shuffle(["no", "."])
@time best_order(test_bag, lm)

(Any["it","is","so","very","good","."],6.803934689506605e-11)

0.113396 seconds (22.27 k allocations: 594.922 KB)


In [40]:
tic()
m=Model(solver=BonminNLSolver())
number_of_constraints=0
_lm = Dict{Tuple{S,S,S},Float64}()
function cached_lm(w1,w2,w3)
    get!(_lm, (w1,w2,w3)) do
       lm(w1,w2,w3)
    end
end


unordered_words  = test_bag
unordered_markers = [START_MARKER1; START_MARKER2; END_MARKER1; unordered_words...]
#Note that this lacks END_MARKER2

nodes = State{Int}[] #Named by word index

node_indexes_for_1st = Dict{Int, Vector{Int}}()
node_indexes_for_2nd = Dict{Int, Vector{Int}}()

function add_node!(ii,jj)
    push!(nodes, (ii,jj))

    node_indexes_for_i_1st = get!(()->Int[], node_indexes_for_1st, ii)
    push!(node_indexes_for_i_1st, length(nodes))

    node_indexes_for_j_2nd = get!(()->Int[], node_indexes_for_2nd, jj) 
    push!(node_indexes_for_j_2nd, length(nodes))
    #println("node:$(length(nodes)) |  $(unordered_markers[ii])($ii), $(unordered_markers[jj])($jj)")
end

add_node!(1, 2) #That is START_MARKER1-> STARTMARKER2

for ii in 1:length(unordered_markers)
    wi = unordered_markers[ii]
    if wi==END_MARKER1 || wi==START_MARKER1 continue end 
        #No node for the transition from END_MARKER1 to anything (not even END_MARKER2 as we do not have to model that as it is a constant prob 1)
        #No transition for START_MARKER1 to anyting but start maker 2
    for jj in [1:length(unordered_markers)]
        if ii==jj continue end
        wj = unordered_markers[jj]
        if wj==START_MARKER1 || wj==START_MARKER2 continue end
        add_node!(ii,jj)
    end
end

@defVar(m, x[1:length(nodes), 1:length(nodes)], Bin)

for class_index in 1:length(unordered_markers)
    w_class =  unordered_markers[class_index]
    if !(w_class==START_MARKER1 || w_class==START_MARKER2)#Not rquired to have either START_MARKER ever occur in second position
        @addConstraint(m, sum{x[ii,jj],ii=1:length(nodes), jj=node_indexes_for_2nd[class_index]}==1)
    end
    
    if !(w_class==END_MARKER1)#Not rquired to have END_MARKER ever occur in fist position
        @addConstraint(m, sum{x[ii,jj], ii=node_indexes_for_1st[class_index],jj=1:length(nodes)}==1)
    end
end


trans_prob = spzeros(length(nodes), length(nodes))
for (from_node_index, from_node) in enumerate(nodes)
    w1 = unordered_markers[from_node[1]]
    w2 = unordered_markers[from_node[2]]
    #If what was in the second state element does not end up in the first state element then it is not allowed.
    can_transition_to = Set(get(node_indexes_for_1st, from_node[2], Int[]))
        #You can transition to any node which has your second element as its first element
    for (to_node_index, to_node) in enumerate(nodes)
        if to_node_index in can_transition_to
            @assert(from_node[2]==to_node[1])
            w3= unordered_markers[to_node[2]]
            tp = cached_lm(w1,w2,w3)
            if tp>0.0
                trans_prob[from_node_index, to_node_index] = tp
                continue
            else
                #Banned as prob zero transitions not allowed
                @addConstraint(m, x[from_node_index,to_node_index]==0)
            end
            
        else
            #It is not a legal transition
            @addConstraint(m, x[from_node_index,to_node_index]==0)
        end
    end
end

iis, jjs, tp_values = findnz(trans_prob)
@setNLObjective(m, Max, prod{max((x[i,j]-1)^2, trans_prob[i,j]), i=iis, j=jjs})
#Note: max((x[i,j]-1)^2, trans_prob[i,j]) is trick for: x[i,j] ? trans_prob[i,j] : 1
toc()#34 seconds
@printval length(unordered_markers)
@printval length(m.linconstr)
trans_prob


elapsed time: 0.

44x44 sparse matrix with 93 Float64 entries:
	[1 ,  3]  =  0.000928671
	[1 ,  4]  =  0.0355816
	[1 ,  5]  =  0.000353157
	[1 ,  6]  =  0.00382368
	[1 ,  7]  =  0.000335717
	[1 ,  8]  =  0.00110307
	[3 ,  9]  =  0.986111
	[16,  9]  =  0.998689
	[28,  9]  =  0.984694
	[34,  9]  =  0.984694
	⋮
	[20, 42]  =  0.00493197
	[32, 42]  =  0.00257591
	[38, 42]  =  0.00180212
	[8 , 43]  =  0.00126222
	[20, 43]  =  0.0042517
	[32, 43]  =  0.00287896
	[38, 43]  =  0.00201413
	[8 , 44]  =  0.00046503
	[20, 44]  =  0.00221088
	[32, 44]  =  0.00106067
	[38, 44]  =  0.000742049

272657926 seconds
length(unordered_markers) = 9
length(m.linconstr) = 1858


In [41]:
solve(m)
println("Objective value: ", getObjectiveValue(m))
x_val = getValue(x)
println("x = ", x_val)
x_iis,x_jjs, _  = findnz(x_val)
zip(x_iis,x_jjs) |> collect

Bonmin 1.8.2 using Cbc 2.9.1 and Ipopt 3.12.1
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1      INFEAS 0.24447294       17 7.277
NLP0014I             2      INFEAS 0.24447294       17 7.191517
Cbc0006I The LP relaxation is infeasible or too expensive

 	"Finished"


Objective value: 0
x = [NaN

1936-element Array{Tuple{Int64,Int64},1}:
 (1,1)  
 (2,1)  
 (3,1)  
 (4,1)  
 (5,1)  
 (6,1)  
 (7,1)  
 (8,1)  
 (9,1)  
 (10,1) 
 (11,1) 
 (12,1) 
 (13,1) 
 ⋮      
 (33,44)
 (34,44)
 (35,44)
 (36,44)
 (37,44)
 (38,44)
 (39,44)
 (40,44)
 (41,44)
 (42,44)
 (43,44)
 (44,44)

 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

In [ ]:
function get_prod(x, iis, jjs)
    net = 1.0
    for i in 1:size(x,1) 
        for j in 1:size(x,2)
            println(i,",", j, " =", x[i,j], " ", trans_prob[i,j])
            net*=max((x[i,j]-1)^2, trans_prob[i,j])
        end
    end
    net
end
values = falses(size(x))
values[1,3] = 1
values[3,6] = 1
values[6,7] = 1
println("----------\n")
get_prod(values, iis,jjs)


In [ ]:
node_indexes_for_1st

In [ ]:
node_indexes_for_2nd

In [ ]:
m

In [ ]:
function get_transition_probs(unordered_words::Vector{S}, languauge_model::Function; beam_width=Inf)
    
    trans_prob = Dict(Tuple{Int,Int},Vector{Float64})
    unordered_tokens = S[unordered_words...; END_MARKER1]
    i_start1 = -1
    i_start2 = 0
    
    trans_prob[(i_start1,i_start2)] = Float64[languauge_model(START_MARKER1,START_MARKER2, wi) for wi inunordered_tokens]
    
    for i1,i2,i3 in permutations(1:length(unordered_words),3)
        w1 = unordered_words[i1]
        w2 = unordered_words[i2]
        w3 = unordered_words[i3]
        
        tps = get!(trans_prob, (i1,i2)) do
            tps=zeros(length(unordered_tokens))
            tps[end] = languauge_model(w1,w2, END_MARKER1)
        end
        tps[i3] = languauge_model(w1,w2, w3)
    end
    trans_prob
end

In [ ]:
for ii in [1:3;5:6]
    println(ii)
end

In [ ]:
[1,1,1,2,3,4,5],3) |> collect

In [ ]:
#Prepare Oracle Results -- no regen
test_set = load("results/data/brown_glove300.jld")["corpus"]
oracle_res = pmap(test_set, err_stop=true) do target_sent
    score=0.0
    sol = shuffle(target_sent)
    (target_sent, sol, score)
end
@save "results/bags/oracle_res.jld" oracle_res

In [ ]:
@everywhere const len_cap = 18
@everywhere raw_bow_res = load("results/bags/oracle_res.jld", "oracle_res")

function process_fold(fold_ii)
    test_bows, test_ground_order, train = fold_split(fold_ii,raw_bow_res)
    
    #Avoid "serialising a pointer" by getting each process to create their own copy of Language model
    #This also means that the language models are independent (as under the hood they are not readonly, readomg them changes them as they have cache)
    r_language_models =Dict([pid=>remotecall(pid, train_language_model, train) for pid in workers()])
    pmap(test_ground_order, test_bows, err_stop=true) do ground_order, bow
        lm = fetch(r_language_models[myid()])
        generated_order, prob = if length(ground_order)<=len_cap
            best_order(bow, lm; beam_width=5)
        else
            (bow, NaN)
        end
        (ground_order, generated_order, prob)
    end
end

jldopen("results/ordered/oracle_orderedv2.jld", "w") do file
    for fold_ii in 1:length(fold_indexes)
        res = process_fold(fold_ii)
        write(file, "fold_$(fold_ii)", res)
    end
end


In [ ]:
1+1

In [ ]:
ordered_res = map(bow_res) do rt
    ground_order, bow, score = rt
    generated_order, prob = best_order(bow, lm; beam_width=5)
    println(generated_order)
    (ground_order, generated_order, prob)
end

In [ ]:
test_bows, test_ground_order, train = fold_split(2,raw_bow_res)
language_model = train_language_model(train)



In [ ]:
@time ord, prob = best_order(test_bows[4110], language_model; beam_width=5)

In [ ]:
res = pmap(test_bow, err_stop=true) do bow
    order, prob = best_order(bow, language_model; beam_width=5)
    (target_sent, sol, score)
end


In [ ]:
5^20

In [ ]:
@pipe test |> map(x->length(x)==50, _) |> find

In [ ]:
@time best_order(test[60], language_model, beam_width=5)

In [ ]:
ground_sents[16]

In [ ]:
training_trigrams = [nltk.trigrams(sent)|>collect for sent in train]


In [ ]:
map(examples) do unordered_words
    order(unordered_words, ASCIIString[])
    end

In [ ]:
short_cases = Bool[length(ws) <=20 for ws in unordered_output]

true_ordered_sents = test_set[short_cases]
ordered_sents_and_probs = pmap(unordered_output[short_cases]) do unordered_words
    order(unordered_words, zeroed_words)
end

In [ ]:
ordered_sents = map(op->op[1], ordered_sents_and_probs, be)

In [ ]:
perfect_matches = Bool[]  
for ii in 1:length(ordered_sents)
    ordered_words = ordered_sents[ii]
    actual_words = true_ordered_sents[ii]
    
    match = ordered_words == actual_words
    push!(perfect_matches, match)
    #println("$ii - $match")
end
mean(perfect_matches)

In [ ]:
@pyimport nltk
@pyimport nltk.translate.bleu_score as nltk_bleu

function bleu_score(candidate, reference)
    reference = reference |> collect
    candidate = candidate |> collect
    
    if reference==candidate #Perfect Match
        1.0
    else
        weights = [1,1,1,1]/4
        nltk_bleu.bleu(Any[reference],candidate, weights)
    end
end

In [ ]:
map(bleu_score, ordered_sents,true_ordered_sents) |> mean


In [ ]:
bleu_score(true_ordered_sents[3], true_ordered_sents[3])

In [ ]:
true_ordered_sents

In [ ]:
ordered_sents

In [ ]:
ordered_sents[50]

In [ ]:
unordered_output[short_cases][eval_cases][50]

In [ ]:
true_ordered_sents[50]

In [ ]:
#x= ["A", "B", "C", "D"]
#y= UTF8String["A", "B", "C", "D"]
x = true_ordered_sents[10]
y=ordered_sents[10] |> collect
pycall(nltk_bleu.bleu, PyAny, Any[x], y, Any[0.25, 0.25, 0.25, 0.25])

In [ ]:
Any[true_ordered_sents[1]]


In [ ]:
nltk_bleu._modified_precision(Any[reference1, reference2, reference3],candidate1, 1)

In [ ]:
@pyimport pdb

In [ ]:
pdb.runcall(nltk_bleu._modified_precision, [reference1, reference2, reference3],candidate1, 1)

In [ ]:
# I think I have to reimplement BLEU in julia as for some reason it does not play nice with PyCall
# Can basically port http://www.nltk.org/_modules/nltk/align/bleu_score.html#bleu

In [ ]:
pycall(nltk_bleu.bleu, Int, candidate1, [reference1], weights)

In [ ]:
@pyimport nltk.util as nltk_util
nltk_util.ngrams(candidate1,2) |> py_collections.Counter

In [ ]:
@pyimport collections as py_collections

In [ ]:
@pyimport nltk.util as nltk_util
ngs = nltk_util.ngrams(candidate1,2)
pycall(py_collections.Counter, PyObject, ngs)

In [ ]:
import nltk